# L1: Textsegmentering och Korpusanalys

Från ett datorperspektiv är en text i första hand en sekvens av tecken, såsom bokstäver och siffror. Innan vi kan bearbeta en text med språkteknologiska verktyg behöver vi dela upp den i lingvistiskt mer meningsfulla enheter, såsom stycken, meningar eller ord. När enheterna är just ord kallas denna segmentering för **tokenisering**. I denna laboration ska ni implementera en enkel tokeniserare för löpande text.

### Läs in råtexten

Texten som ni ska jobba med i den första uppgiften är en artikel från svenska Wikipedia: [Gustav III](https://sv.wikipedia.org/wiki/Gustav_III). På en Wikipedia-sida förekommer inte bara text utan även andra data, såsom bilder och tabeller. Innan ni kan tokenisera texten skulle ni därför egentligen först behöva extrahera den från sidan. För just den här uppgiften har vi dock redan gjort detta för er. Den extraherade texten finns i filen `text1.txt`.

För att läsa in den extraherade texten i Python definierar vi en hjälpfunktion `read_data()`. Funktionen öppnar den angivna filen och returnerar dess innehåll som en lista med textrader, en sträng per rad. I textfilen avslutas varje rad med nyrad-tecknet (`\n`); detta tecken tas bort med hjälp av [`rstrip()`](https://docs.python.org/3/library/stdtypes.html#str.rstrip).

In [1]:
def read_data(filename):
    with open(filename) as f:
        return [line.rstrip() for line in f]

Nu kan ni läsa in råtexten:

In [2]:
text1 = read_data('text1.txt')

Nästa cellen skriver ut de första 50 raderna i texten:

In [ ]:
print(text1[:50])

### Läs in guldstandarden

Till råtexten finns även en guldstandardtokenisering. Denna tokenisering följer de regler som används i [Stockholm–Umeå Corpus (SUC)](https://spraakbanken.gu.se/swe/resurs/suc3), en standardkorpus för svenska. Filen med guldstandardtokeniseringen innehåller alla token i råtexten, ett token per rad.

In [4]:
gold1 = read_data('gold1.txt')

Titta på denna guldstandard och försök att förstå de principer den bygger på. De flesta token är vanliga ord eller skiljetecken, men notera att förkortningar behandlas som ett token.

In [ ]:
print(gold1[:50])

## Mellanrumsbaserad tokenisering

Nästa cell innehåller en mycket enkel tokeniserare:

In [6]:
def tokenize_ws(lines):
    tokens = []
    for line in lines:
        for token in line.split():
            tokens.append(token)
    return tokens

Denna funktion tar in en lista med textrader, bryter upp varje rad vid mellanrum (*whitespace*) genom att anropa metoden [`split()`](https://docs.python.org/3.5/library/stdtypes.html#str.split) och samlar in de resulterande strängarna i en lista `tokens`.

### Jämför tokeniseringen med guldstandarden

Provkör tokeniseraren på de 50 första raderna i texten:

In [ ]:
print(tokenize_ws(text1[:50]))

Jämför denna tokenisering med guldstandarden. Vilka skillnader finns?

De flesta skillnaderna kommer att vara fall av **undersegmentering**, där tokeniseraren har missat att dela upp ett token. Motsatsen till detta är **översegmentering**, där tokeniseraren har delat upp det som egentligen borde ha varit ett token i flera.

För att undersöka skillnaderna kan ni använda funktionen `diff()` från labmodulen. Denna funktion tar två argument, en lista med guldstandardtoken och en lista med automatiskt predicerade token, och returnerar en ny lista som beskriver skillnaderna mellan dessa två tokeniseringarna på ett kompakt sätt. Följande kommando t.ex. ger er de första tio skillnaderna:

In [ ]:
import lab1

lab1.diff(gold1, tokenize_ws(text1))[:10]

Listan innehåller  par vars första komponent är en sekvens av token som förekommer i guldstandarden men inte förekommer i den automatiska tokeniseringen, och vars andra komponent är en sekvens som förekommer i den automatiska tokeniseringen men inte i guldstandarden. Följande kodbit skriver ut listan så att den blir mera läsbar; den skriver även ut antalet token i respektive sekvens:

In [ ]:
# Formatera en lista med token.
def fmt_tokens(tokens):
    return ' '.join(tokens) + ' ({})'.format(len(tokens))

# Skriv ut information om avvikande delsekvenser.
print('Gold tokens'.ljust(40), 'Predicted tokens'.ljust(40))
print()
for gold_tokens, pred_tokens in lab1.diff(gold1, tokenize_ws(text1)):
    print(fmt_tokens(gold_tokens).ljust(40), fmt_tokens(pred_tokens).ljust(40))

Undersök skillnaderna mellan guldstandarden och den mellanrumsbaserade tokeniseringen. Leta efter exempel för över- och undersegmentering. 

### Beräkna precision och täckning

I Uppgift&nbsp;1 gjorde ni vad man kallar en *kvalitativ* utvärderingen av er tokeniserare. Ett sätt att göra en mera kvantitativ utvärdering är att räkna ut dess **precision** och dess **täckning** (recall). Precision är definierad som andelen korrekt identifierade token bland alla token som tokeniseraren producerar. Ett token räknas som korrekt identifierat när det täcker samma teckenpositioner i texten som ett token i guldstandarden. Recall är definierad som andelen korrekt identifierade token bland alla token som finns i guldstandarden. För att beräkna dessa värden kan ni använda följande kodcell:

In [ ]:
tokens_ws = tokenize_ws(text1)

print('Errors: {}'.format(lab1.n_errors(gold1, tokens_ws)))
print('Precision: {:.4f}'.format(lab1.precision(gold1, tokens_ws)))
print('Recall: {:.4f}'.format(lab1.recall(gold1, tokens_ws)))

### Tokenisering baserad på reguljära uttryck

I andra delen av denna laboration ska ni ersätta den enkla mellanrumsbaserade tokeniseringen med en mera avancerad tokenisering baserad på **reguljära uttryck**. Om ni känner att ni behöver öva mer på reguljära uttryck innan ni kan göra uppgiften kom ihåg de rekommenderade webbsidor: [Regex Golf](https://alf.nu/RegexGolf) och [Regex 101](https://regex101.com).

Innan ni kan använda reguljära uttryck i Python måste ni först ladda den relevanta modulen:

In [11]:
import re

En enkel tokeniserare baserad på reguljära uttryck ser ut så här:

In [12]:
def tokenize_re(regex, lines):
    output = []
    for line in lines:
        for match in re.finditer(regex, line):
            output.append(match.group(0))
    return output

Denna funktion hittar alla längsta, icke-överlappande förekomster av mönstret `regex` i raden `line` och returnerar dem som en lista. Raden skannas från vänster till höger och de matchande delsträngarna returneras i samma ordning.

För att återskapa och köra den mellanrumsbaserade tokeniseraren med hjälp av reguljära uttryck kan ni använda följande kod:

In [ ]:
# Reguljärt uttryck som tokeniseraren ska använda. TODO: Modifiera den för att få bättre resultat. 
regex = r'\S+'

tokens_re = tokenize_re(regex, text1)

print('Errors: {}'.format(lab1.n_errors(gold1, tokens_re)))
print('Precision: {:.4f}'.format(lab1.precision(gold1, tokens_re)))
print('Recall: {:.4f}'.format(lab1.recall(gold1, tokens_re)))

# För att felsöka regexen kan det vara bra att kommentera in följande rad:
# lab1.diff(gold1, tokens_re)

**Uppgift 1**: Hitta ett reguljärt uttryck som eliminerar så många skillnader mellan guldstandarden och den automatiska tokeniseringen som möjligt. Använd era insikter från Uppgift&nbsp;1. Er färdiga tokeniserare bör komma upp i minst 99,5% precision och täckning.

### Utvärdera den förbättrade tokeniseraren på en ny text

**Uppgift 2**: **Utvärdera er tokeniserare på en andra artikel från svenska Wikipedia:
[Katarina II av Ryssland](https://sv.wikipedia.org/wiki/Katarina_II_av_Ryssland). (Hon var förresten kusin till Gustav&nbsp;III.) Ange även här antal fel, precision och täckning. 

Råtext och guldstandardtokeniseringen laddar ni så här:

In [14]:
text2 = read_data('text2.txt')
gold2 = read_data('gold2.txt')

# TODO: Skriv kod för att utvärdera tokeniseraren på den nya texten här.

## Korpusanalys

Nu har ni en tokeniserare och är redo för att skapa en egen liten korpus och göra en statistisk analys på den! 

### Bygga Korpusen

**Uppgift 3**: Ladda ner minst 10 (längre) Wikipediaartiklar och extrahera råtexten. Använd kodskeletten i cellerna nedan och ersätt Kapybara-artikeln med era valda artiklar. 

In [21]:
# TODO: Ladda ner 10 artiklar genom att lägga till dem i listan `urls`.

import requests

urls = ["https://sv.wikipedia.org/wiki/Kapybara"]

for url in urls: 
    response = requests.get(url)

    with open("corpus.html", 'a', encoding='utf-8') as file:
        file.write(response.text)

Nu ska vi bli av med html-annoteringarna och bara spara texten i paragraferna: 

In [22]:
# TODO: Extrahera råtexten från alla 10 artiklar.

from bs4 import BeautifulSoup

with open("corpus.html", 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

paragraphs = soup.find_all('p')
cleaned_text = '\n'.join([para.get_text() for para in paragraphs])

with open("corpus.txt", 'w', encoding='utf-8') as file:
    file.write(cleaned_text.strip())

Nice! Nu har ni skapat en egen liten korpus och sparat resultatet i `corpus.txt`. Nu behöver vi tokenisera texten för att kunna göra en ordfrekvensanalys: 

**Uppgift 4**: Använd eran tokeniserare på korpusen. För att göra detta, anropa funktionen `tokenize_re` från tidigare. 

In [ ]:
# TODO: Tokenisera texterna. 

###  Ordfrekvensanalys

Nu ska ni räkna alla ord i dem 10 artiklarna. Använd gärna en [Counter](https://docs.python.org/3/library/collections.html#counter-objects) från Python's *collections* library på listan för att enkelt räkna dem. 

**Uppgift 5**: Använd tokenlistan från uppgift 4 och spara ordfrekvenserna över hela korpusen i en Counter. 

In [ ]:
from collections import Counter

# TODO: Skapa en Counter som innehåller ordfrekvenserna.

**Uppgift 6** Titta på resultatet. Ni kan kolla på dem mest frekventa orden med Counter's [`most_common()`](https://docs.python.org/3/library/collections.html#collections.Counter.most_common) funktion. Tycker ni att korpusen uppfyller [Zipf’s law](https://en.wikipedia.org/wiki/Zipf%27s_law) och/eller [Zipf’s brevity law](https://en.wikipedia.org/wiki/Brevity_law) som vi pratade om i föreläsningen? Varför (inte)? 

TODO: Svara på frågan och reflektera resultatet i 2-3 meningar. 

In [ ]:
# Titta närmare på Countern i den här cellen.

Skriv analysen i den här cellen. 